In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import spdiags
from scipy.sparse import csr_matrix
from scipy.integrate import solve_ivp
from scipy.linalg import lu, solve
from scipy.sparse.linalg import gmres
from scipy.sparse.linalg import bicgstab
from scipy.linalg import lu, solve_triangular
from scipy.fftpack import fft2, ifft2
from scipy.sparse.linalg import spsolve

### creating matrices

m = 64 # N value in x and y directions
n = m * m # total size of matrix
deltax = 20/m
deltaS = (20/m) **2

e0 = np.zeros((n, 1)) # vector of zeros
e1 = np.ones((n, 1)) # vector of ones

e2 = np.copy(e1) # copy the one vector
e4 = np.copy(e0) # copy the zero vector

for j in range(1, m+1):
    e2[m*j-1] = 0 # overwrite every m^th value with zero
    e4[m*j-1] = 1 # overwirte every m^th value with one
    
# Shift to correct positions
e3 = np.zeros_like(e2)
e3[1:n] = e2[0:n-1]
e3[0] = e2[n-1]

e5 = np.zeros_like(e4)
e5[1:n] = e4[0:n-1]
e5[0] = e4[n-1]

diagonals = [e1.flatten(), e1.flatten(), e5.flatten(),
e2.flatten(), -4 * e1.flatten(), e3.flatten(),
e4.flatten(), e1.flatten(), e1.flatten()]
offsets = [-(n-m), -m , -(m-1), -1, 0, 1, m - 1, m , (n-m)]
matA = spdiags(diagonals, offsets, n, n).tocsr()


A = matA.toarray()
A = A / deltaS

matA = matA/deltaS
A_w_0_changed = matA.toarray()
A_w_0_changed[0,0] = 2

### dy

diagonalsC = [e5.flatten(), -1 * e2.flatten(), e3.flatten(), -1 * e4.flatten()]
offsetsC = [-(m-1),-1,1,m-1]
matC = spdiags(diagonalsC, offsetsC, n, n)


C = matC.toarray() 
C = C/ (2 * deltax)

matC = matC/ (2 * deltax)

### dx

diagonalsB = [e1.flatten()/  (2 * deltax), -1 * e1.flatten()/ (2 * deltax), e1.flatten()/ (2 * deltax), -1 * e1.flatten()/ (2 * deltax)]
offsetsB = [-(n-m),-m,m, n-m]
matB = spdiags(diagonalsB, offsetsB, n, n)

B = matB.toarray()


       
    

In [2]:
### fft solve

def fft(w0, tspan, n):
    v = 0.001
    nx, ny = n, n
    N = nx * ny
    
    ## spectral k values
    kx = (2 * np.pi / Lx) * np.concatenate((np.arange(0,nx/2), np.arange(-nx/2, 0)))
    ky = (2 * np.pi / Ly) * np.concatenate((np.arange(0,ny/2), np.arange(-ny/2, 0)))
    kx[0] = 1e-6
    ky[0] = 1e-6
    
    KX, KY = np.meshgrid(kx,ky)
    K = KX**2 + KY**2
    
    def fft_rhs(t, wv, nx, ny, N, K, matA, matB, matC):
        wrv = np.real(wv)
        wfft = fft2(w0)
        psi_ft = -wfft / K
        psiV = np.real(ifft2(psi_ft)).reshape((nx **2,))
        
        return (v * np.dot(A, wrv) - np.dot(B,psiV)) * (np.dot(C, wrv)) + (np.dot(C, psiV)) * (np.dot(B, wrv))
            
    
    sol_ft = solve_ivp(fft_rhs, (tspan[0],tspan[-1]), wrv, method='RK45', t_eval=tspan, args = (nx, ny, N, K, matA, matB, matC))
    return sol_ft.y

In [3]:
## initial conditions & calling A1
n = 64
v = 0.001
Lx,Ly = 20,20
nx, ny = n, n
N = nx * ny
tspan = np.linspace(0,4, 8 + 1)


X = np.linspace(-10,10,n+1)
Y = np.linspace(-10,10,n+1)
x = X[:n]
y = Y[:n]
x1, y1 = np.meshgrid(x, y)

w0 = 1 * np.exp((-x1**2) -((y1**2)/20)) + 1j * np.zeros((nx,ny))
wv = w0.reshape(nx **2,)
wrv = np.real(wv)

A1 = fft(w0, tspan, n)
print(A1.shape)

(4096, 9)


In [4]:
### A / b

n = 64
l = 4
dl = 0.5
v = 0.001
tspan = np.linspace(0,l, 8 + 1)

X = np.linspace(-10,10,n+1)
Y = np.linspace(-10,10,n+1)

x = X[:n]
y = Y[:n]

x1, y1 = np.meshgrid(x, y)

w = np.exp((-x1**2) -((y1**2)/20))

### A/ B


def a_b(w, tspan, n):
    matA[0,0] = 2
    
    A_w_0_changed = matA.toarray()
    A_w_0_changed[0,0] = 2
    
    nx, ny = n, n
    N = nx * ny
    wVec = w.reshape(n**2,)
    
    def a_b_rhs(t, wVec, nx, ny, matA, matB, matC):
        psi_ab = spsolve(matA, wVec)
        
        return (v * np.dot(A_w_0_changed, wVec) - np.dot(B,psi_ab)) * (np.dot(C, wVec)) + (np.dot(C, psi_ab)) * (np.dot(B, wVec))
            
   
    sol_ab = solve_ivp(a_b_rhs, (tspan[0],tspan[-1]), wVec, method='RK45', t_eval=tspan, args = (nx, ny, matA, matB, matC))
    return sol_ab.y
       

In [5]:
A2 = a_b(w, tspan, n)
print(A2.shape)

(4096, 9)


In [6]:

### lu decomp

n = 64
l = 4
dl = 0.5
v = 0.001
tspan = np.linspace(0,l, 8 + 1)
X = np.linspace(-10,10,n+1)
Y = np.linspace(-10,10,n+1)

x = X[:n]
y = Y[:n]

x1, y1 = np.meshgrid(x, y)

w0_lu = np.exp((-x1**2) -((y1**2)/20))

P, L, U = lu(A)

def L_U(w0_lu, tspan, n):
    A_w_0_changed = matA.toarray()
    A_w_0_changed[0,0] = 2
    
    wVec_lu = w0_lu.reshape(n**2,)
   
    def lu_rhs(t, wVec_lu, P, L, U, matA, matB, matC):
        
        Pb = np.dot(P, wVec_lu)
        SOL = solve_triangular(L, Pb, lower=True)
        psi_lu = solve_triangular(U, SOL)
    
        return (v * np.dot(A_w_0_changed, wVec_lu) - np.dot(B,psi_lu)) * (np.dot(C, wVec_lu)) + (np.dot(C, psi_lu)) * (np.dot(B, wVec_lu))
    
    sol = solve_ivp(lu_rhs, (tspan[0],tspan[-1]), wVec_lu, method='RK45', t_eval=tspan, args = (P, L, U, matA, matB, matC))
    return sol.y

In [7]:
A3 = L_U(w0_lu, tspan, n)
print(A3.shape)

(4096, 9)
